<a href="https://colab.research.google.com/github/Weinihsiang/Pytorch/blob/main/cv2library.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np

def resize_nearest_neighbor(image, new_size):
    old_height, old_width = image.shape[:2]
    new_height, new_width = new_size

    # Calculate scaling ratios
    height_ratio = old_height / new_height
    width_ratio = old_width / new_width

    # Generate coordinates for new image pixels
    new_y = np.arange(new_height) * height_ratio
    new_x = np.arange(new_width) * width_ratio

    # Round the coordinates to the nearest integer
    new_y = np.round(new_y).astype(int)
    new_x = np.round(new_x).astype(int)

    # Ensure that the coordinates are within the image bounds
    new_y = np.clip(new_y, 0, old_height - 1)
    new_x = np.clip(new_x, 0, old_width - 1)

    # Reshape coordinates for indexing
    new_y = new_y.reshape((new_height, 1))
    new_x = new_x.reshape((1, new_width))

    # Index the original image with the new coordinates
    resized_image = image[new_y, new_x]

    return resized_image


nearest neighbor

In [ ]:
import numpy as np

def resize_bilinear(image, new_size):
    old_height, old_width = image.shape[:2]
    new_height, new_width = new_size

    # Calculate scaling ratios
    height_ratio = old_height / new_height
    width_ratio = old_width / new_width

    # Generate coordinates for new image pixels
    y = np.arange(new_height) * height_ratio
    x = np.arange(new_width) * width_ratio

    # Calculate the indices of the four nearest pixels for each new coordinate
    y_floor = np.floor(y).astype(int)
    x_floor = np.floor(x).astype(int)
    y_ceil = np.clip(np.ceil(y).astype(int), 0, old_height - 1)
    x_ceil = np.clip(np.ceil(x).astype(int), 0, old_width - 1)

    # Calculate the fractional part of each coordinate
    y_fraction = y - y_floor
    x_fraction = x - x_floor

    # Interpolate values for each channel using bilinear interpolation
    interpolated = np.zeros((new_height, new_width, image.shape[2]), dtype=np.uint8)
    for channel in range(image.shape[2]):
        top_left = image[y_floor, x_floor, channel]
        top_right = image[y_floor, x_ceil, channel]
        bottom_left = image[y_ceil, x_floor, channel]
        bottom_right = image[y_ceil, x_ceil, channel]

        interpolated[:, :, channel] = (1 - y_fraction) * ((1 - x_fraction) * top_left + x_fraction * top_right) + \
                                       y_fraction * ((1 - x_fraction) * bottom_left + x_fraction * bottom_right)

    return interpolated


bilinear method

In [ ]:
import numpy as np

def resize_bicubic(image, new_size):
    old_height, old_width = image.shape[:2]
    new_height, new_width = new_size

    # Calculate scaling ratios
    height_ratio = old_height / new_height
    width_ratio = old_width / new_width

    # Generate coordinates for new image pixels
    y = np.arange(new_height) * height_ratio
    x = np.arange(new_width) * width_ratio

    # Calculate the indices and fractional parts of the coordinates
    y_floor = np.floor(y).astype(int)
    x_floor = np.floor(x).astype(int)
    y_fraction = y - y_floor
    x_fraction = x - x_floor

    # Apply bicubic interpolation to each channel
    interpolated = np.zeros((new_height, new_width, image.shape[2]), dtype=np.uint8)
    for channel in range(image.shape[2]):
        for i in range(new_height):
            for j in range(new_width):
                i_floor = y_floor[i]
                j_floor = x_floor[j]

                p = np.zeros((4, 4))
                for m in range(-1, 3):
                    for n in range(-1, 3):
                        p[m+1, n+1] = image[
                            np.clip(i_floor + m, 0, old_height - 1),
                            np.clip(j_floor + n, 0, old_width - 1),
                            channel
                        ]

                interpolated[i, j, channel] = bicubic_interpolation(p, y_fraction[i], x_fraction[j])

    return interpolated


def bicubic_interpolation(p, y_fraction, x_fraction):
    a = np.array([
        [-0.5, 1.5, -1.5, 0.5],
        [1, -2.5, 2, -0.5],
        [-0.5, 0, 0.5, 0],
        [0, 1, 0, 0]
    ])

    fractional_matrix = np.array([1, x_fraction, x_fraction ** 2, x_fraction ** 3])
    interpolated_value = np.dot(fractional_matrix, np.dot(a, np.dot(p, a.T)))

    return np.clip(interpolated_value, 0, 255).astype(np.uint8)


cubic interpolation

In [ ]:
import numpy as np

import numpy as np

def lanczos_kernel(x, a):
    if x == 0:
        return 1.0
    elif -a < x < a:
        return a * np.sin(np.pi * x) * np.sin(np.pi * x / a) / (np.pi * np.pi * x * x)
    else:
        return 0.0

def resize_lanczos4(image, new_size):
    new_height, new_width = new_size
    height, width = image.shape[:2]
    resized = np.zeros((new_height, new_width, image.shape[2]), dtype=np.uint8)

    y_ratio = height / new_height
    x_ratio = width / new_width

    for i in range(new_height):
        for j in range(new_width):
            y = (i + 0.5) * y_ratio
            x = (j + 0.5) * x_ratio

            y_floor = int(y)
            x_floor = int(x)
            a = 3

            result = np.zeros(image.shape[2], dtype=np.float32)
            for m in range(-a + 1, a):
                for n in range(-a + 1, a):
                    y_fraction = y - y_floor
                    x_fraction = x - x_floor
                    weight = lanczos_kernel(y_fraction - m, a) * lanczos_kernel(x_fraction - n, a)
                    if 0 <= y_floor + m < height and 0 <= x_floor + n < width:
                        result += weight * image[y_floor + m, x_floor + n]

            resized[i, j] = np.clip(result, 0, 255).astype(np.uint8)

    return resized


Lanczos interpolation with a kernel size of 4 using NumPy

In [ ]:
from google.colab import files
from PIL import Image

# Upload the image file
uploaded = files.upload()

# Get the uploaded image file name
image_filename = list(uploaded.keys())[0]

# Load the image
image = Image.open(image_filename)

# Convert the image to NumPy array
image_array = np.array(image)

# Resize the image using custom interpolation methods
resized_nearest = resize_nearest_neighbor(image_array, (2000, 3000))
resized_bilinear = resize_bilinear(image_array, (2000, 3000))
resized_bicubic = resize_bicubic(image_array, (2000, 3000))
resized_lanczos = resize_lanczos4(image_array, (2000, 3000))

# Convert the resized images back to PIL Image
resized_nearest = Image.fromarray(resized_nearest)
resized_bilinear = Image.fromarray(resized_bilinear)
resized_bicubic = Image.fromarray(resized_bicubic)
resized_lanczos = Image.fromarray(resized_lanczos)

# Display or save the resized images as desired
resized_nearest.show()
resized_bilinear.show()
resized_bicubic.show()
resized_lanczos.show()


Saving piano.png to piano.png


NameError: ignored

In [ ]:
nodes = int(input("How many nodes are there? "))
number = [[0] * (nodes+1) for _ in range(nodes+1)]
numberexist = [0]*(nodes+1)
height = [0]*(nodes+1)
max = [-1]*(nodes+1)

for i in range(nodes):
    list = input().split()
    for j in range(int(list[0])):
        number[i+1][j] = int(list[j+1])
        numberexist[number[i+1][j]]+=1
print(number)
for k in range(nodes+1):
    for meow in range(nodes+1):
        l = 0
        while(number[meow][l]!=0):
            if height[int(number[meow][l])]>=max[meow]:
                max[meow] = height[int(number[meow][l])]
                height[meow] = height[int(number[meow][l])]+1
            l +=1
    if numberexist[k] ==0 and k != 0:
        print(k)
print(height)

How many nodes are there? 9
1 6
3 5 3 8
0
2 1 7
1 9
0
1 2
0
0
[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [6, 0, 0, 0, 0, 0, 0, 0, 0, 0], [5, 3, 8, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [1, 7, 0, 0, 0, 0, 0, 0, 0, 0], [9, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [2, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0]]
4
[0, 1, 2, 0, 4, 1, 0, 3, 0, 0]


In [ ]:
pip install pycraigslist

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.4/75.4 kB 4.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 653.6/653.6 kB 10.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 74.5/74.5 kB 2.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 2.8 MB/s eta 0:00:00
  Created wheel for cchardet: filename=cchardet-2.1.7-cp310-cp310-linux_x86_64.whl size=249355 sha256=f9611349ef7f8173d5d157ee5eb4a14131b2cfda7bf5fde370865642770eb34f
  Stored in directory: /root/.cache/pip/wheels/ee/e0/ab/e01326f15c59438d080b1496dbab8091e952ec72f35e3c437e
  Created wheel for bs4: filename=bs4-0.0.1-py3-none-any.whl size=1256 sha256=dac901cbb5b6ee756107b19c706e8e347ed221a08a6ab4c2be0187b5617bbce2
  Stored in directory: /root/.cache/pip/wheels/25/42/45/b773edc52acb16cd2db4cf1a0b47117e2f69bb4eb300ed0e70
Successfully built cchardet bs4


In [ ]:
import pycraigslist

print(pycraigslist.housing.get_categories())

{'apa': 'apartments / housing for rent', 'swp': 'housing swap', 'off': 'office & commercial', 'prk': 'parking & storage', 'rea': 'real estate', 'reb': 'real estate - by dealer', 'reo': 'real estate - by owner', 'roo': 'rooms & shares', 'sub': 'sublets & temporary', 'vac': 'vacation rentals', 'hou': 'wanted: apts', 'rew': 'wanted: real estate', 'sha': 'wanted: room/share', 'sbw': 'wanted: sublet/temp'}


In [ ]:
word = "APPLEPPLE"
word[::-1]

'ELPPELPPA'

In [ ]:
# -*- coding: utf-8 -*-
import scrapy


class EbaySpider(scrapy.Spider):

	name = "ebay"
	allowed_domains = ["ebay.com"]
	start_urls = ["https://www.ebay.com"]

	# Allow a custom parameter (-a flag in the scrapy command)
	def __init__(self, search="nintendo switch console"):
		self.search_string = search

	def parse(self, response):
		# Extrach the trksid to build a search request
		trksid = response.css("input[type='hidden'][name='_trksid']").xpath("@value").extract()[0]

		# Build the url and start the requests
		yield scrapy.Request("http://www.ebay.com/sch/i.html?_from=R40&_trksid=" + trksid +
							 "&_nkw=" + self.search_string.replace(' ','+') + "&_ipg=200",
							 callback=self.parse_link)

		print('Hello Michael')

	# Parse the search results
	def parse_link(self, response):
		# Extract the list of products
		results = response.xpath('//div/div/ul/li[contains(@class, "s-item" )]')

		# Extract info for each product
		for product in results:
			name = product.xpath('.//*[@class="s-item__title"]//text()').extract_first()
			# Sponsored or New Listing links have a different class
			if name == None:
				name = product.xpath('.//*[@class="s-item__title s-item__title--has-tags"]/text()').extract_first()
				if name == None:
					name = product.xpath('.//*[@class="s-item__title s-item__title--has-tags"]//text()').extract_first()
			if name == 'New Listing':
				name = product.xpath('.//*[@class="s-item__title"]//text()').extract()[1]

			# If this get a None result
			if name == None:
				name = "ERROR"

			price = product.xpath('.//*[@class="s-item__price"]/text()').extract_first()
			status = product.xpath('.//*[@class="SECONDARY_INFO"]/text()').extract_first()
			seller_level = product.xpath('.//*[@class="s-item__etrs-text"]/text()').extract_first()
			location = product.xpath('.//*[@class="s-item__location s-item__itemLocation"]/text()').extract_first()
			product_url = product.xpath('.//a[@class="s-item__link"]/@href').extract_first()

			# Set default values
			stars = 0
			ratings = 0

			stars_text = product.xpath('.//*[@class="clipped"]/text()').extract_first()
			if stars_text: stars = stars_text[:3]
			ratings_text = product.xpath('.//*[@aria-hidden="true"]/text()').extract_first()
			if ratings_text: ratings = ratings_text.split(' ')[0]

			summary_data = {
							"Name":name,
							"Status":status,
							#"Seller_Level":seller_level,
							#"Location":location,
							"Price":price,
							"Stars":stars,
							"Ratings":ratings,
							"URL": product_url
							}

			# Go to the product details page
			data = {'summary_data': summary_data}
			print(data)
			# print(scrapy.Request(product_url, meta=data, callback=self.parse_product_details))

		# Get the next page
		next_page_url = response.xpath('//*/a[@class="x-pagination__control"][2]/@href').extract_first()

		# The last page do not have a valid url and ends with '#'
		if next_page_url == None or str(next_page_url).endswith("#"):
			self.log("eBay products collected successfully !!!")
		else:
			print('\n'+'-'*30)
			print('Next page: {}'.format(next_page_url))
			yield scrapy.Request(next_page_url, callback=self.parse_link)


	# Parse details page for each product
	def parse_product_details(self, response):

		# Get the summary data
		data = response.meta['summary_data']

		# Add more data from details page
		data['UPC'] = response.xpath('//h2[@itemprop="gtin13"]/text()').extract_first()

		yield data



In [ ]:
from scrapy.crawler import CrawlerProcess
from scrapy.utils.project import get_project_settings


def run_spider():
    process = CrawlerProcess(get_project_settings())
    process.crawl(EbaySpider)  # Replace with your actual spider class
    process.start()

run_spider()


INFO:scrapy.utils.log:Scrapy 2.10.0 started (bot: scrapybot)
2023-08-26 18:23:15 [scrapy.utils.log] INFO: Scrapy 2.10.0 started (bot: scrapybot)
INFO:scrapy.utils.log:Versions: lxml 4.9.3.0, libxml2 2.10.3, cssselect 1.2.0, parsel 1.8.1, w3lib 2.1.2, Twisted 22.10.0, Python 3.10.12 (main, Jun 11 2023, 05:26:28) [GCC 11.4.0], pyOpenSSL 23.2.0 (OpenSSL 3.1.2 1 Aug 2023), cryptography 41.0.3, Platform Linux-5.15.109+-x86_64-with-glibc2.35
2023-08-26 18:23:15 [scrapy.utils.log] INFO: Versions: lxml 4.9.3.0, libxml2 2.10.3, cssselect 1.2.0, parsel 1.8.1, w3lib 2.1.2, Twisted 22.10.0, Python 3.10.12 (main, Jun 11 2023, 05:26:28) [GCC 11.4.0], pyOpenSSL 23.2.0 (OpenSSL 3.1.2 1 Aug 2023), cryptography 41.0.3, Platform Linux-5.15.109+-x86_64-with-glibc2.35
INFO:scrapy.addons:Enabled addons:
[]
2023-08-26 18:23:15 [scrapy.addons] INFO: Enabled addons:
[]
INFO:scrapy.crawler:Overridden settings:
{}
2023-08-26 18:23:15 [scrapy.crawler] INFO: Overridden settings:
{}
DEBUG:scrapy.utils.log:Using rea

ReactorNotRestartable: ignored